In [2]:
import random
import pandas as pd
import numpy as np

In [330]:
sensor_file        = 'sensors10'
cov_file               = 'cov10'
hypothesis_file = 'hypothesis10'

# Generate Sensors

In [331]:
grid_len = 10
all_locations = []
for i in range(grid_len):
    for j in range(grid_len):
        all_locations.append((i, j))

num_sensor = 20
sensors = random.sample(all_locations, num_sensor)
sensors

[(7, 3),
 (8, 5),
 (7, 9),
 (1, 3),
 (4, 3),
 (7, 8),
 (8, 3),
 (0, 7),
 (8, 0),
 (7, 7),
 (7, 5),
 (9, 4),
 (0, 0),
 (1, 8),
 (1, 1),
 (0, 4),
 (6, 6),
 (5, 3),
 (3, 8),
 (5, 6)]

In [332]:
ipsn = pd.read_csv('sensor_from_ipsn', delimiter=' ', header=None)
stds = ipsn[2]
all_stds = np.unique(stds.values)
all_stds

array([0.31961585, 0.41428093, 0.41543918, 0.42026762, 0.45186382,
       0.51815505, 0.6606852 , 0.7535492 , 1.04563019, 1.12004327,
       1.43466257, 1.52048246, 1.83061047, 1.85433775, 1.86264196,
       2.02253412, 2.34168195, 2.57483698])

In [333]:
std_costs = []
for sen in sensors:
    std_costs.append((random.choice(all_stds), 1)) # homogeneous
    
for std_cost in std_costs:
    print(std_cost)

(1.8626419592629413, 1)
(0.31961585322723524, 1)
(1.8306104706098087, 1)
(2.341681953932289, 1)
(1.8306104706098087, 1)
(1.1200432714506303, 1)
(0.4202676150502109, 1)
(2.02253412315468, 1)
(0.4142809311761813, 1)
(2.574836982024013, 1)
(1.045630193236076, 1)
(1.045630193236076, 1)
(2.574836982024013, 1)
(0.4202676150502109, 1)
(1.1200432714506303, 1)
(1.1200432714506303, 1)
(1.4346625681183898, 1)
(0.4202676150502109, 1)
(0.5181550542899774, 1)
(1.8626419592629413, 1)


In [334]:
with open(sensor_file, 'w') as f:
    for index, sensor in enumerate(sensors):
        #print(index, sensor)
        f.write('{} {} {} {}\n'.format(sensor[0], sensor[1], std_costs[index][0], std_costs[index][1]))

# Generate Covariance

In [335]:
with open(cov_file, 'w') as f:
    grid_len = 20   # the number of sensors
    cov = np.zeros((grid_len, grid_len))
    for i in range(grid_len):
        for j in range(grid_len):
            if i == j:
                cov[i, j] = std_costs[i][0] ** 2
            f.write('{} '.format(cov[i, j]))
        f.write('\n')

# Generate Hypothesis

#### Frist, deal with latitute and longtitue and SPLAT software

In [336]:
'''
left_up    = (-73.111278, 40.872335)
right_up   = (-73.099383, 40.872335)
left_down  = (-73.111278, 40.863346)  # here is (0, 0)
right_down = (-73.099383, 40.863346)
grid_len   = 10
long_step  = (right_up[0] - left_up[0])/grid_len
lat_step   = (right_up[1] - right_down[1])/grid_len

trans_long_lat = []  # in the end, print in (lat, long)
for i in range(grid_len):
    for j in range(grid_len):
        x = left_down[0] + i*long_step
        y = left_down[1] + j*lat_step
        trans_long_lat.append((x, y))
'''

'\nleft_up    = (-73.111278, 40.872335)\nright_up   = (-73.099383, 40.872335)\nleft_down  = (-73.111278, 40.863346)  # here is (0, 0)\nright_down = (-73.099383, 40.863346)\ngrid_len   = 10\nlong_step  = (right_up[0] - left_up[0])/grid_len\nlat_step   = (right_up[1] - right_down[1])/grid_len\n\ntrans_long_lat = []  # in the end, print in (lat, long)\nfor i in range(grid_len):\n    for j in range(grid_len):\n        x = left_down[0] + i*long_step\n        y = left_down[1] + j*lat_step\n        trans_long_lat.append((x, y))\n'

In [337]:
'''
with open('lat_long', 'w') as f:
    for tran in trans_long_lat:
        f.write('{}, {}, {}\n'.format(tran[1], tran[0], 30.0))
'''

"\nwith open('lat_long', 'w') as f:\n    for tran in trans_long_lat:\n        f.write('{}, {}, {}\n'.format(tran[1], tran[0], 30.0))\n"

#### Second, remove the outliers (pathloss < 80)

In [338]:
'''
grid_len = 10
hypo_template = 'tx_{}_pathloss.txt'
hypo = None
for i in range(1, 101):
    filename = hypo_template.format(i)
    hypo = pd.read_csv(filename, delimiter=' ', header=None)
    flag = False
    for j in range(grid_len):
        for k in range(grid_len):
            if hypo.iloc[j, k] < 84:
                print(i, hypo.iloc[j, k])
                #flag = True
                #print(i, j, hypo.iloc[j, k], end=' --> ')
                #hypo.iat[j, k] = 109 + np.random.rand()*2
                #print(hypo.iloc[j, k])
    #if flag:
    #    hypo.to_csv(filename, sep=' ', index=False, header=False)
'''

"\ngrid_len = 10\nhypo_template = 'tx_{}_pathloss.txt'\nhypo = None\nfor i in range(1, 101):\n    filename = hypo_template.format(i)\n    hypo = pd.read_csv(filename, delimiter=' ', header=None)\n    flag = False\n    for j in range(grid_len):\n        for k in range(grid_len):\n            if hypo.iloc[j, k] < 84:\n                print(i, hypo.iloc[j, k])\n                #flag = True\n                #print(i, j, hypo.iloc[j, k], end=' --> ')\n                #hypo.iat[j, k] = 109 + np.random.rand()*2\n                #print(hypo.iloc[j, k])\n    #if flag:\n    #    hypo.to_csv(filename, sep=' ', index=False, header=False)\n"

#### Create hypothesis file

In [339]:
'''
sensors = []
std_costs = []
with open('sensors', 'r') as f:
    lines = f.readlines()
    for line in lines:
        line = line.split()
        sensors.append((int(line[0]), int(line[1])))
        std_costs.append((float(line[2]), int(line[3])))
'''

"\nsensors = []\nstd_costs = []\nwith open('sensors', 'r') as f:\n    lines = f.readlines()\n    for line in lines:\n        line = line.split()\n        sensors.append((int(line[0]), int(line[1])))\n        std_costs.append((float(line[2]), int(line[3])))\n"

In [340]:
grid_len = 10
hypo_template = 'tx_{}_pathloss.txt'
hypo = None
hypothesis_file = open(hypothesis_file ,'w')
for i in range(1, 101):
    if i%10 == 0:
        print(i, end=' ')
    hypo = pd.read_csv(hypo_template.format(i), delimiter=' ', header=None)
    trans_x = (i-1)//grid_len
    trans_y = (i-1)%grid_len
    for index, sensor in enumerate(sensors):
        sen_x = sensor[0]
        sen_y = sensor[1]
        std = std_costs[index][0]
        mean = 25 - hypo.iloc[sen_y, sen_x]  # cellular tower power - pathloss
        print(trans_x, trans_y, sen_x, sen_y, mean, std, file=hypothesis_file)
hypothesis_file.close()

10 20 30 40 50 60 70 80 90 100 

In [3]:
array = np.ones((4, 4))

In [4]:
array

array([[1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.]])

In [5]:
np.power(10, np.array(array)/20)

array([[1.12201845, 1.12201845, 1.12201845, 1.12201845],
       [1.12201845, 1.12201845, 1.12201845, 1.12201845],
       [1.12201845, 1.12201845, 1.12201845, 1.12201845],
       [1.12201845, 1.12201845, 1.12201845, 1.12201845]])